# PM2.5 Exploratory Data Analysis (EDA)

**Station:** 10T — เคหะชุมชนคลองจั่น, เขตบางกะปิ, กทม.  
**Data:** PM2.5(2024).xlsx (Train) / PM2.5(2025).xlsx (Test)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (14, 5)
plt.rcParams['font.size'] = 12
sns.set_style('whitegrid')

# Load data
STATION = '10T'

df24 = pd.read_excel('../data/raw/PM2.5(2024).xlsx', sheet_name='Data')
df25 = pd.read_excel('../data/raw/PM2.5(2025).xlsx', sheet_name='DATA')

# Clean Date column
df24['Date'] = pd.to_datetime(df24['Date'], errors='coerce')
df25['Date'] = pd.to_datetime(df25['Date'], errors='coerce')
df24 = df24.dropna(subset=['Date'])
df25 = df25.dropna(subset=['Date'])

# Extract station data
train = df24[['Date', STATION]].copy()
train.columns = ['date', 'pm25']
train['pm25'] = pd.to_numeric(train['pm25'], errors='coerce')

test = df25[['Date', STATION]].copy()
test.columns = ['date', 'pm25']
test['pm25'] = pd.to_numeric(test['pm25'], errors='coerce')

print(f"=== Station {STATION} ===")
print(f"Train (2024): {train.shape[0]} days, missing: {train['pm25'].isna().sum()}")
print(f"Test  (2025): {test.shape[0]} days, missing: {test['pm25'].isna().sum()}")
print(f"\nTrain stats:\n{train['pm25'].describe()}")
print(f"\nTest stats:\n{test['pm25'].describe()}")

## 1. PM2.5 Time Series Plot

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16, 8), sharex=False)

# 2024 (Train)
axes[0].plot(train['date'], train['pm25'], color='steelblue', linewidth=0.8)
axes[0].axhline(y=37.5, color='orange', linestyle='--', label='Thailand Standard (37.5 µg/m³)')
axes[0].set_title(f'Station {STATION} — PM2.5 Daily Values (2024, Train)')
axes[0].set_ylabel('PM2.5 (µg/m³)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2025 (Test)
axes[1].plot(test['date'], test['pm25'], color='coral', linewidth=0.8)
axes[1].axhline(y=37.5, color='orange', linestyle='--', label='Thailand Standard (37.5 µg/m³)')
axes[1].set_title(f'Station {STATION} — PM2.5 Daily Values (2025, Test)')
axes[1].set_ylabel('PM2.5 (µg/m³)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/pm25_time_series.png', dpi=150, bbox_inches='tight')
plt.show()

## 2. Distribution of PM2.5 Values

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(train['pm25'].dropna(), bins=30, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].axvline(train['pm25'].mean(), color='red', linestyle='--', label=f'Mean={train["pm25"].mean():.1f}')
axes[0].set_title('PM2.5 Distribution — 2024 (Train)')
axes[0].set_xlabel('PM2.5 (µg/m³)')
axes[0].set_ylabel('Frequency')
axes[0].legend()

axes[1].hist(test['pm25'].dropna(), bins=25, color='coral', alpha=0.7, edgecolor='black')
axes[1].axvline(test['pm25'].mean(), color='red', linestyle='--', label=f'Mean={test["pm25"].mean():.1f}')
axes[1].set_title('PM2.5 Distribution — 2025 (Test)')
axes[1].set_xlabel('PM2.5 (µg/m³)')
axes[1].set_ylabel('Frequency')
axes[1].legend()

plt.tight_layout()
plt.savefig('../results/pm25_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Monthly Seasonal Pattern

In [ ]:
train_monthly = train.copy()
train_monthly['month'] = train_monthly['date'].dt.month

fig, ax = plt.subplots(figsize=(12, 5))
train_monthly.boxplot(column='pm25', by='month', ax=ax)
ax.set_title(f'Station {STATION} — PM2.5 by Month (2024)')
ax.set_xlabel('Month')
ax.set_ylabel('PM2.5 (µg/m³)')
ax.axhline(y=37.5, color='red', linestyle='--', alpha=0.5, label='Thailand Standard')
plt.suptitle('')  # Remove auto-title
ax.legend()
plt.tight_layout()
plt.savefig('../results/pm25_monthly_boxplot.png', dpi=150, bbox_inches='tight')
plt.show()

print("Dry season (Nov-Apr) vs Wet season (May-Oct):")
dry = train_monthly[train_monthly['month'].isin([1,2,3,4,11,12])]['pm25']
wet = train_monthly[train_monthly['month'].isin([5,6,7,8,9,10])]['pm25']
print(f"  Dry season: mean={dry.mean():.1f}, std={dry.std():.1f}")
print(f"  Wet season: mean={wet.mean():.1f}, std={wet.std():.1f}")

## 4. Autocorrelation — Lag Feature Analysis

In [ ]:
# Create lag features for correlation analysis
lag_df = train[['pm25']].dropna().copy()
for lag in [1, 2, 3, 5, 7, 14]:
    lag_df[f'lag_{lag}'] = lag_df['pm25'].shift(lag)
lag_df = lag_df.dropna()

# Correlation heatmap
corr = lag_df.corr()
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr, annot=True, fmt='.3f', cmap='RdBu_r', center=0, ax=ax)
ax.set_title(f'Station {STATION} — Autocorrelation (PM2.5 vs Lag Features)')
plt.tight_layout()
plt.savefig('../results/pm25_autocorrelation.png', dpi=150, bbox_inches='tight')
plt.show()

print("Lag correlations with current PM2.5:")
for lag in [1, 2, 3, 5, 7, 14]:
    r = lag_df['pm25'].corr(lag_df[f'lag_{lag}'])
    print(f"  Lag {lag:2d}: r = {r:.4f}")

## 5. Missing Value Pattern (All Stations, 2024)

In [ ]:
# Missing value analysis across all stations in 2024
df24_clean = df24.dropna(subset=['Date']).copy()
stations = [c for c in df24_clean.columns if c != 'Date']

missing_pct = {}
for s in stations:
    col = pd.to_numeric(df24_clean[s], errors='coerce')
    missing_pct[s] = col.isna().sum() / len(df24_clean) * 100

missing_sorted = dict(sorted(missing_pct.items(), key=lambda x: x[1], reverse=True))

fig, ax = plt.subplots(figsize=(16, 5))
bars = ax.bar(range(len(missing_sorted)), list(missing_sorted.values()), color='steelblue', alpha=0.7)
ax.set_xlabel('Station Index')
ax.set_ylabel('Missing %')
ax.set_title(f'Missing Value % by Station (2024) — {len(stations)} stations')
ax.axhline(y=5, color='red', linestyle='--', alpha=0.5, label='5% threshold')
ax.legend()
plt.tight_layout()
plt.savefig('../results/missing_values_pattern.png', dpi=150, bbox_inches='tight')
plt.show()

# Station 10T
print(f"\nStation 10T missing: {missing_pct.get('10T', 'N/A'):.1f}%")
print(f"Stations with >10% missing: {sum(1 for v in missing_pct.values() if v > 10)}")
print(f"Stations with <2% missing: {sum(1 for v in missing_pct.values() if v < 2)}")

## 6. Summary

**Key Findings:**
- PM2.5 shows strong **seasonality**: high in dry season (Nov-Apr), low in wet season (May-Oct)
- Strong **autocorrelation**: lag-1 correlation is very high → lag features are useful predictors
- **Missing data** is minimal for station 10T (~3% in 2024) → forward-fill is sufficient
- Distribution is **right-skewed** → extreme high PM2.5 events are rarer but important to predict
- 2025 test data has slightly **higher mean** than 2024 → possible distribution shift to monitor